In [24]:
using DataFrames, CSV, Rotations, GeometryBasics

## Load Transforms

In [2]:
# Create the dict to map user_id to transform dataframe
transforms = Dict()

# Load the transforms
for filename in readdir(raw"..\Data\Dataframes\Transforms", join=true)
    # Get the dataframe from the file
    df = DataFrame(CSV.File(filename))
    # Get the id of the user
    id = parse(Int, match(r"(\d{2,3})_transforms.csv$", filename)[1])
    # Store dataframe into dictionary
    transforms[id] = df
end

In [7]:
show(transforms[102])

13533×14 DataFrame
   Row │ c_pos_x      c_pos_y      c_pos_z       c_rot_x      c_rot_y      c_r ⋯
       │ Float64      Float64      Float64       Float64      Float64      Flo ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │  3.82586      3.62586      0.0           0.0          0.0         -0. ⋯
     2 │ -0.0          1.0          0.000890538   0.082156    -0.329123     0.
     3 │  0.111715    -0.0586747   -0.988158      0.0145977    0.0794296   -0.
     4 │ -0.0852581    0.11534     -0.0588556    -0.987909     0.00155286   0.
     5 │ -0.0343132   -0.0806783    0.117827     -0.058851    -0.988        0. ⋯
     6 │ -0.0156151   -0.034185    -0.0786815     0.114219    -0.0563988   -0.
     7 │ -0.0226061   -0.0146794   -0.0340413    -0.0761304    0.110792    -0.
     8 │  0.983151    -0.0214697   -0.0141649    -0.0351364   -0.0748463    0.
   ⋮   │      ⋮            ⋮            ⋮             ⋮            ⋮           ⋱
 13527 │ -0.0493905  

## Process data
Condense position coords into single Point3f columns

Condense quaternion components into single quaternion column, also handle handedness

In [27]:
# Just look at the first user for now
user_num = collect(keys(transforms))[1]

# Function to convert the handedness of the rotations from Unity to normal
converthandedness(x,y,z,w) = (w,x,z,y)
# Functions to create quaternions and Points and name the new column
h_post(x) = (; :head_position => Point3f(x...))
h_quat(x) = (; :head_orientation => QuatRotation(converthandedness(x...)...))
c_post(x) = (; :cntr_position => Point3f(x...))
c_quat(x) = (; :cntr_orientation => QuatRotation(converthandedness(x...)...))

# New Dict to hold the transformed data
condensed_transforms = Dict()
# Iterate through all the users
for key in keys(transforms)
    # Construct the dataframe
    condensed_transforms[key] = let 
        # Create a 3D point column for headset position
        h_pos_data = transforms[key][:,Between(:h_pos_x, :h_pos_z)]
        h_pos = select(
            h_pos_data,
            AsTable(:) => ByRow(h_post) => AsTable
        )
        # Create a quaternion column for headset rotation
        h_rot_data = transforms[key][:,Between(:h_rot_x, :h_rot_w)]
        h_rot = select(
            h_rot_data,
            AsTable(:) => ByRow(h_quat) => AsTable
        )
        # Create a 3D point column for controller position
        c_pos_data = transforms[key][:,Between(:c_pos_x, :c_pos_z)]
        c_pos = select(
            c_pos_data,
            AsTable(:) => ByRow(c_post) => AsTable
        )
        # Create a quaternion column for the controller rotation
        c_rot_data = transforms[key][:,Between(:c_rot_x, :c_rot_w)]
        c_rot = select(
            c_rot_data,
            AsTable(:) => ByRow(c_quat) => AsTable
        )
        # Put all the columns together, and return
        hcat(h_pos, h_rot, c_pos, c_rot)
    end
end

condensed_transforms[88]

Row,head_position,head_orientation,cntr_position,cntr_orientation
,Point…,QuatRota…,Point…,QuatRota…
1,"Float32[-0.0, 1.0, 0.0]",[0.0 0.0 0.0; -0.0 0.0 0.0; 0.0 -0.0 0.0],"Float32[3.82827, 3.62827, 0.0]",[0.0 0.0 0.0; -0.0 0.0 0.0; 0.0 -0.0 0.0]
2,"Float32[-0.205649, 0.626851, 0.749204]",[0.869867 -0.462285 0.172117; 0.210676 0.663656 0.717758; -0.446035 -0.588093 0.674685],"Float32[-0.0, 1.0, 0.00109844]",[-0.988832 -0.148602 -0.0113507; 0.136542 -0.872785 -0.468617; 0.0597308 -0.464933 0.883328]
3,"Float32[-0.0588141, -0.205649, 0.626851]",[0.990145 -0.0632864 0.124934; -0.0703684 -0.996106 0.0531071; 0.121086 -0.0613751 -0.990743],"Float32[0.104904, 0.0400868, -0.986772]",[-0.883005 0.00494665 -0.469337; -0.00717949 0.999685 0.0240438; 0.469308 0.0246004 -0.882692]
4,"Float32[0.192555, -0.0588141, -0.205649]",[-0.17743 0.0993626 0.979105; 0.0199419 -0.994323 0.104521; 0.983931 0.0380703 0.174441],"Float32[-0.1181, 0.105502, 0.0403461]",[0.999527 0.0287475 -0.0108849; -0.00367039 -0.239956 -0.970777; -0.0305193 0.970358 -0.239737]
5,"Float32[-0.772436, 0.192555, -0.0588141]",[-0.911205 -0.365333 -0.190361; -0.251833 0.128298 0.959229; -0.326015 0.921993 -0.208908],"Float32[-0.0215214, -0.118967, 0.106582]",[-0.996598 -0.0188537 -0.0802363; 0.0215333 -0.999234 -0.0326639; -0.079559 -0.0342805 0.996241]
6,"Float32[-0.00997893, -0.772436, 0.192555]",[0.129532 0.234411 0.963469; -0.381881 0.908499 -0.169695; -0.915089 -0.34595 0.207197],"Float32[-0.0251266, -0.022235, -0.117863]",[-0.88733 -0.21828 -0.406202; -0.185256 0.975401 -0.119466; 0.422287 -0.0307548 -0.90594]
7,"Float32[0.0161283, 0.0295338, -0.335227]",[-0.497384 -0.520597 0.693966; 0.261298 0.672885 0.692061; -0.827244 0.525552 -0.198651],"Float32[0.0373564, -0.0192985, -0.031811]",[0.990409 0.110732 -0.0826378; -0.124422 0.974844 -0.184927; 0.0600815 0.193435 0.979272]
8,"Float32[-0.992906, 0.0167115, 0.0194246]",[0.573892 0.386857 0.721796; 0.352064 0.679233 -0.643967; -0.739391 0.623686 0.253608],"Float32[0.698916, 0.0364127, -0.0190458]",[-0.694271 0.202438 0.690657; -0.711224 -0.339941 -0.615306; 0.110221 -0.9184 0.37999]
9,"Float32[0.0367144, -0.992671, 0.0165761]",[0.148039 -0.983462 -0.104343; 0.988971 0.146731 0.0201497; -0.00450618 -0.106175 0.994337],"Float32[-0.176112, 0.60044, 0.0344072]",[-0.39079 0.522703 -0.757671; -0.0303908 0.81535 0.57817; 0.919978 0.248969 -0.302745]


## Save Processed Transforms

In [28]:
for key in keys(condensed_transforms)
    CSV.write("../Data/Dataframes/CondensedTransforms/$(key)_transforms.csv", condensed_transforms[key])
end